This notebook is to exemplify the downloading from the database and cache into disk. 

In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

We first load the libraries necessary. We need to use pyfftw to define zeros_aligned arrays, and pyJHTDB for downloading. 

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

import SOAPtdb

Then we need to initialize the associated variables, including domain, pyJHTDB and MPI variables. 

In [3]:
%%px

comm = MPI.COMM_WORLD; rank = comm.Get_rank(); nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# pyJHTDB parameters
    
Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']
auth_key = "com.gmail.jhelsas-b854269a"
 
# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:7] 
n_proc = 8
rank = 0


In [4]:
%%px
## Initialize the velocity field having a size of (lx,ly,lz)
vx=ft.zeros_aligned((nx,ny,nz), dtype='float32')
vy=ft.zeros_aligned((nx,ny,nz), dtype='float32')
vz=ft.zeros_aligned((nx,ny,nz), dtype='float32')

if(rank==0):
    print("shape = "+str(vx.shape))

[stdout:7] shape = (128, 1024, 1024)


In [6]:
%%px
# Populate velocity field from the Database
comm.Barrier(); t1=MPI.Wtime()
SOAPtdb.loadvel(vx,vy,vz,nx,ny,nz,rank)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Load velocity field cost: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Load velocity field cost: 236.87 seconds


Once downloaded, we can cache the data on disk, as being done below. The tipical times required are pretty long, around 900-1100 seconds, which amounts to 15 to 20 minutes. 

In [8]:
%%px

folder = "/home/idies/workspace/scratch"
filename = "kun-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))

[stdout:7] Caching the data: 316.56 seconds
